## Import Packages

In [101]:
# import upstream regulator module
import sys
code_path = '../../network_bio_toolkit'
sys.path.append(code_path)
import Upstream
reload(Upstream)

# heat propagation and clustering
import Heat
reload(Heat)

# gsea
import PrepGsea
reload(PrepGsea)

# this notebook
from IPython.display import Javascript, display
from ipywidgets import widgets, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Text

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+2)'))

%matplotlib inline

In [15]:
def checkbox_maker(descriptions):
    options_dict = {description: widgets.Checkbox(description=description, style = {'description_width': 'initial'}, value=False) for description in descriptions}
    options = [options_dict[description] for description in descriptions]
    options_widget = widgets.VBox(options, layout={'overflow': 'scroll'})
    multi_select = widgets.VBox([options_widget])
    return multi_select

def get_values(checkboxes):
    selected_pipelines = [widget.description for widget in checkboxes.children[0].children if widget.value]
    return selected_pipelines

In [121]:
pipeline_form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between'
    )
pipeline_form_items = [
    Box([Label(value='Pick your pipelines:'), pipeline_checkbox], layout = pipeline_form_item_layout)
]
pipeline_form = Box(pipeline_form_items, layout=Layout(
        display='flex',
        flex_flow='column',
        border='solid 2px',
        align_items='stretch',
        width='45%'
))
display(pipeline_form)

A Jupyter Widget

In [119]:
pipelines = get_values(pipeline_checkbox)
pipelines

[u'GSEA']

In [51]:
children = []
if 'Upstream Regulator Analysis' in pipelines:
    URA_form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between'
    )

    URA_form_items = [
        Box([Label(value='Gene type'), Dropdown(options=['gene_symbol', 'entrez'])], layout = URA_form_item_layout),
        Box([Label(value='Species'), Dropdown(options=['human', 'mouse'])], layout = URA_form_item_layout),
        Box([Label(value='TF Filepath'), Text()], layout = URA_form_item_layout),
        Box([Label(value='Background Network Filepath'), Text()], layout = URA_form_item_layout),
        Box([Label(value='DEG filepath'), Text()], layout = URA_form_item_layout)
    ]

    URA_form = Box(URA_form_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 2px',
            align_items='stretch',
            width='60%'
    ))
    children.append(URA_form)

if 'Heat Propagation and Clustering' in pipelines:
    HC_form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between'
    )

    HC_form_items = [
        Box([Label(value='Gene type'), Dropdown(options=['gene_symbol', 'entrez'])], layout = HC_form_item_layout),
        Box([Label(value='Species'), Dropdown(options=['human', 'mouse'])], layout = HC_form_item_layout),
        Box([Label(value='DEG filepath'), Text()], layout = HC_form_item_layout),
        Box([Label(value='Background Network Filepath'), Text()], layout = HC_form_item_layout)
    ]

    HC_form = Box(HC_form_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 2px',
            align_items='stretch',
            width='60%'
    ))
    children.append(HC_form)

if 'GSEA' in pipelines:
    GSEA_form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between'
    )

    ['gmt_file', 'expression_file', 'meta_file', 'output_dir']

    GSEA_form_items = [
        Box([Label(value='GMT Filepath'), Text()], layout = GSEA_form_item_layout),
        Box([Label(value='Expression Filepath'), Text()], layout = GSEA_form_item_layout),
        Box([Label(value='Meta Filepath'), Text()], layout = GSEA_form_item_layout),
        Box([Label(value='Output Directory'), Text()], layout = GSEA_form_item_layout)
    ]

    GSEA_form = Box(GSEA_form_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 2px',
            align_items='stretch',
            width='60%'
    ))
    children.append(GSEA_form)

In [122]:
accordion = widgets.Accordion(children=children)
for i in range(len(pipelines)):
    accordion.set_title(i, pipelines[i])
display(accordion)

# URA
# ../../TF_databases/TF_database_URA.csv
# ../../background_networks/9606.protein.actions.v10.5.txt
# ../../DEG_databases/DE_Coeff_OAvsNormal_OAvsNormal_20171215.csv

# Heat Prop and Cluster
# ../../DEG_databases/DE_CoeffspaceFlight - groundControl_glds48_20180312.csv
# ../../background_networks/10090.protein.actions.v10.5.txt

# GSEA
# ../../gseapy_databases/h.all.v6.1.symbols.gmt
# ../../gseapy_databases/RNA_cpm_all_samples.tsv
# ../../gseapy_databases/dHEP_metadata.csv
# ../../gseapy_databases/mouse_liver_output

A Jupyter Widget

In [107]:
for i in range(len(pipelines)):
    
    if pipelines[i] == 'Upstream Regulator Analysis':
        URA_gene_type = accordion.children[i].children[0].children[1].value
        URA_species = accordion.children[i].children[1].children[1].value
        URA_TF_file = accordion.children[i].children[2].children[1].value
        URA_background_file = accordion.children[i].children[3].children[1].value
        URA_DEG_file = accordion.children[i].children[4].children[1].value
        
    elif pipelines[i] == 'Heat Propagation and Clustering':
        HC_gene_type = accordion.children[i].children[0].children[1].value
        HC_species = accordion.children[i].children[1].children[1].value
        HC_DEG_file = accordion.children[i].children[2].children[1].value
        HC_background_file = accordion.children[i].children[3].children[1].value
        
    else:
        gmt_file = accordion.children[i].children[0].children[1].value
        expression_file = accordion.children[i].children[1].children[1].value
        meta_file = accordion.children[i].children[2].children[1].value
        output_dir = accordion.children[i].children[3].children[1].value
        

In [111]:
# decide user preferences

if (drop_down.value == 'Upstream Regulator Analysis') | (drop_down.value == 'Heat Propagation and Clustering'):
    
    gene_symbol_widget = widgets.RadioButtons(
        options=['gene_symbol', 'entrez'],
        value='gene_symbol',
        description='gene type:',
        disabled=False
    )
    
    species_widget = widgets.RadioButtons(
        options=['human', 'mouse'],
        value='human',
        description='species:',
        disabled=False
    )
    
    if (drop_down.value == 'Upstream Regulator Analysis'): headers = ['TF filepath', 'background network filepath', 'DEG filepath']
    elif (drop_down.value == 'Heat Propagation and Clustering'): headers = ['background network filepath', 'DEG filepath']  
    else: headers = ['gmt_file', 'expression_file', 'meta_file', 'output_dir']

    controls_widget_array = []
    for header in headers:
        text = widgets.Text(description = header, style = {'description_width': 'initial'})
        display(text)
        controls_widget_array.append(text)
    
    display(gene_symbol_widget)
    display(species_widget)

NameError: name 'drop_down' is not defined

In [109]:
# decide which funcitons and displays to run

if (drop_down.value == 'Upstream Regulator Analysis'):
    
    print 'Pick which functions you would like to run for ' + str(drop_down.value).upper() + ':'
    descriptions = ['TF-target enrichment', 'TF enrichment', 'z-score', 'top values', 'rank genes', 'display TF network', 'print to file']
    options_dict = {description: widgets.Checkbox(description=description, style = {'description_width': 'initial'}, value=False) for description in descriptions}
    options = [options_dict[description] for description in descriptions]
    options_widget = widgets.VBox(options, layout={'overflow': 'scroll'})
    multi_select = widgets.VBox([options_widget])
    display(multi_select) 

elif (drop_down.value == 'Heat Propagation and Clustering'):
    
    print 'Pick which functions you would like to run for ' + str(drop_down.value).upper() + ':'
    descriptions = ['Heat propagation display', 'Clustering WITHOUT radial positioning', 'Clustering WITH radial positioning']
    options_dict = {description: widgets.Checkbox(description=description, style = {'description_width': 'initial'}, value=False) for description in descriptions}
    options = [options_dict[description] for description in descriptions]
    options_widget = widgets.VBox(options, layout={'overflow': 'scroll'})
    multi_select = widgets.VBox([options_widget])
    display(multi_select) 

NameError: name 'drop_down' is not defined

In [51]:
selected_options = [widget.description for widget in multi_select.children[0].children if widget.value]
print selected_options

print gene_symbol_widget.value
print species_widget.value

[u'Upstream Regulator Analysis']


NameError: name 'gene_symbol_widget' is not defined

In [52]:
# declare your file paths

if (drop_down.value == 'Upstream Regulator Analysis'): headers = ['TF filepath', 'background network filepath', 'DEG filepath']
elif (drop_down.value == 'Heat Propagation and Clustering'): headers = ['background network filepath', 'DEG filepath']  
else: headers = ['gmt_file', 'expression_file', 'meta_file', 'output_dir']

controls_widget_array = []
for header in headers:
    text = widgets.Text(description = header, style = {'description_width': 'initial'})
    display(text)
    controls_widget_array.append(text)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [113]:
[box.value for box in controls_widget_array]
print controls_widget_array[0].value
print controls_widget_array[1].value
print gene_symbol_widget.value
print species_widget.value

1
2


In [111]:
if (drop_down.value == 'Upstream Regulator Analysis'): 
    ura = Upstream.Upstream(gene_type = gene_symbol_widget.value, species = species_widget.value)
    
elif (drop_down.value == 'Heat Propagation and Clustering'): 
    heat = Heat.Heat(gene_type = gene_symbol_widget.value, species = species_widget.value) 
    
else: 
    if u'' in [box.value for box in controls_widget_array]: print 'One of your pathways is blank'
    gsea_prepper = PrepGsea.PrepGsea(gmt_file = controls_widget_array[0].value,
                        expression_file = controls_widget_array[1].value,
                        meta_file = controls_widget_array[2].value,
                        output_dir = controls_widget_array[3].value)
    

IOError: File C does not exist